#### STAC Rucio - Non-Deterministic Rucio Storage Elements demo

This notebook demonstrates the workflow for using the stac_rucio package to register data at a non-deterministic Rucio Storage Element, and the basic management of replication rules to move data from that location to other Rucio Storage Elements ( RSE ).

The RSE "EODC-DATA" was prepared as our non-deterministic Rucio Storage Element, and data will be replicated to "DESY-DCACHE".

In [ ]:
from stac_rucio.client import StacClient

# STAC Rucio client, connects to Rucio and defines a replication target for the client.
stac_rucio = StacClient()

In [2]:
from pystac_client import Client

# STAC client connection, with a modifier for updating items on the fly.
eodc = Client.open(
  "https://stac.eodc.eu/api/v1",
  modifier=stac_rucio.rucio_item
)

NameError: name 'stac_rucio' is not defined

In [1]:
# For the demo we'll define a single aoi, but two different time periods.
aoi = {
  "type": "Polygon",
  "coordinates": [
    [
      [9.184570312500002, 43.628123412124616],
      [25.543212890625004, 43.628123412124616],
      [25.543212890625004, 49.830896288288976],
      [9.184570312500002, 49.830896288288976],
      [9.184570312500002, 43.628123412124616]
    ]
  ]
}

december = {"interval": ["2022-12-01T00:00:00Z", "2022-12-31T00:00:00Z"]}
november = {"interval": ["2022-11-01T00:00:00Z", "2022-11-30T00:00:00Z"]}

In [4]:
# The SENTINEL2_MFCOVER collection at EODC was duplicated with additional metadata for experimenting with the STAC Rucio integration.

items_december = eodc.search(filter_lang="cql2-json", filter={
  "op": "and",
  "args": [
    {"op": "s_intersects", "args": [{"property": "geometry"}, aoi]},
    {"op": "anyinteracts", "args": [{"property": "datetime"}, december]},
    {"op": "=", "args": [{"property": "collection"}, "RUCIO_SENTINEL2_MFCOVER"]}
  ]}
)

items_november = eodc.search(filter_lang="cql2-json", filter={
  "op": "and",
  "args": [
    {"op": "s_intersects", "args": [{"property": "geometry"}, aoi]},
    {"op": "anyinteracts", "args": [{"property": "datetime"}, november]},
    {"op": "=", "args": [{"property": "collection"}, "RUCIO_SENTINEL2_MFCOVER"]}
  ]}
)

In [ ]:
december_list = [ item for item in items_december.items() ]
december_list[0]

In [ ]:
november_list = [ item for item in items_november.items() ]
november_list[0]

In [ ]:
# Create replicas carries out the initial registration of the target asset at the specified RSE.
stac_rucio.create_replicas(items=november_list, rse="EODC-DATA", target="mfcover")

In [ ]:
# Create replication rules generates the rules that instruct rucio that a request has been made to copy data from one RSE to another.
stac_rucio.create_replication_rules(items=november_list, dst_rse="DESY-DCACHE")

In [ ]:
# Availability gets the total number of rules for an item list, and the sum of each state.
stac_rucio.replication_availability(items=november_list, rse="DESY-DCACHE")

In [ ]:
stac_rucio.replication_availability(items=december_list, rse="DESY-DCACHE")

In [ ]:
stac_rucio.download_available(items=december_list[:2], rse="DESY-DCACHE")